# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Load data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Define features

In [3]:
features = ['RhythmScore', 'AudioLoudness', 'VocalContent', 'AcousticQuality', 
            'InstrumentalScore', 'LivePerformanceLikelihood', 'MoodScore', 
            'TrackDurationMs', 'Energy']
X = train[features]
y = train['BeatsPerMinute']
X_test = test[features]

# Feature engineering: Log-transform skewed features

In [4]:
X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
X['TrackDurationMs'] = np.log1p(X['TrackDurationMs'])
X_test['TrackDurationMs'] = np.log1p(X_test['TrackDurationMs'])

C:\Users\nishi\AppData\Local\Temp\ipykernel_21292\3028610528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_21292\3028610528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_21292\3028610528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

# Split data (80/20 for holdout and meta-training)

In [5]:
X_train_full, X_holdout, y_train_full, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features

In [6]:
scaler = StandardScaler()
X_train_full_scaled = scaler.fit_transform(X_train_full)
X_holdout_scaled = scaler.transform(X_holdout)
X_test_scaled = scaler.transform(X_test)

# Train Base Models

# Linear Regression

In [7]:
lr = LinearRegression()
lr.fit(X_train_full_scaled, y_train_full)
lr_holdout_pred = lr.predict(X_holdout_scaled)
lr_rmse = np.sqrt(mean_squared_error(y_holdout, lr_holdout_pred))
print(f"Linear RMSE: {lr_rmse:.4f}")

Linear RMSE: 26.4440


# Random Forest (your best params)

In [8]:
rf = RandomForestRegressor(n_estimators=150, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1)
rf.fit(X_train_full_scaled, y_train_full)
rf_holdout_pred = rf.predict(X_holdout_scaled)
rf_rmse = np.sqrt(mean_squared_error(y_holdout, rf_holdout_pred))
print(f"RF RMSE: {rf_rmse:.4f}")

RF RMSE: 26.4398


# XGBoost with tuned params

In [9]:
xgb_model = xgb.XGBRegressor(n_estimators=120, learning_rate=0.05, max_depth=5, random_state=42)
xgb_model.fit(X_train_full_scaled, y_train_full)
xgb_holdout_pred = xgb_model.predict(X_holdout_scaled)
xgb_rmse = np.sqrt(mean_squared_error(y_holdout, xgb_holdout_pred))
print(f"XGBoost RMSE: {xgb_rmse:.4f}")

XGBoost RMSE: 26.4439


# LightGBM with tuned params

In [10]:
lgb_model = lgb.LGBMRegressor(n_estimators=250, learning_rate=0.03, max_depth=7, random_state=42, n_jobs=-1)
lgb_model.fit(X_train_full_scaled, y_train_full)
lgb_holdout_pred = lgb_model.predict(X_holdout_scaled)
lgb_rmse = np.sqrt(mean_squared_error(y_holdout, lgb_holdout_pred))
print(f"LightGBM RMSE: {lgb_rmse:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 419331, number of used features: 9
[LightGBM] [Info] Start training from score 119.056554
LightGBM RMSE: 26.4417


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


# Generate meta-features (predictions) for stacking

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
meta_X_train = np.zeros((len(X_train_full_scaled), 4))
meta_X_holdout = np.zeros((len(X_holdout_scaled), 4))
meta_X_test = np.zeros((len(X_test_scaled), 4))

for train_idx, val_idx in kf.split(X_train_full_scaled):
    X_train_fold, X_val_fold = X_train_full_scaled[train_idx], X_train_full_scaled[val_idx]
    y_train_fold = y_train_full.iloc[train_idx]
    
    lr.fit(X_train_fold, y_train_fold)
    rf.fit(X_train_fold, y_train_fold)
    xgb_model.fit(X_train_fold, y_train_fold)
    lgb_model.fit(X_train_fold, y_train_fold)
    
    meta_X_train[val_idx] = np.column_stack((
        lr.predict(X_val_fold),
        rf.predict(X_val_fold),
        xgb_model.predict(X_val_fold),
        lgb_model.predict(X_val_fold)
    ))
meta_X_holdout = np.column_stack((
    lr.predict(X_holdout_scaled),
    rf.predict(X_holdout_scaled),
    xgb_model.predict(X_holdout_scaled),
    lgb_model.predict(X_holdout_scaled)
))
meta_X_test = np.column_stack((
    lr.predict(X_test_scaled),
    rf.predict(X_test_scaled),
    xgb_model.predict(X_test_scaled),
    lgb_model.predict(X_test_scaled)
))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003343 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 335464, number of used features: 9
[LightGBM] [Info] Start training from score 119.062610


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 335465, number of used features: 9
[LightGBM] [Info] Start training from score 119.054608


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003904 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 335465, number of used features: 9
[LightGBM] [Info] Start training from score 119.039780


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 335465, number of used features: 9
[LightGBM] [Info] Start training from score 119.078083


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 335465, number of used features: 9
[LightGBM] [Info] Start training from score 119.047690


c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


# Train Meta-Model (Linear Regression)

In [12]:
meta_model = LinearRegression()
meta_model.fit(meta_X_train, y_train_full)
meta_holdout_pred = meta_model.predict(meta_X_holdout)
meta_rmse = np.sqrt(mean_squared_error(y_holdout, meta_holdout_pred))
print(f"Meta-Model Holdout RMSE: {meta_rmse:.4f}")

Meta-Model Holdout RMSE: 26.4391


# Generate final test predictions

In [13]:
test_pred = meta_model.predict(meta_X_test)
submission['BeatsPerMinute'] = test_pred
submission.to_csv('submission_ensemble_stacking.csv', index=False)
print("Submission file created: submission_ensemble_stacking.csv")

Submission file created: submission_ensemble_stacking.csv
